# 전원MinMax

In [12]:
from pandas import DataFrame,read_csv,read_excel,merge,melt,pivot_table
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sys
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from factor_analyzer.factor_analyzer import calculate_kmo

In [2]:
origin=read_excel('행정동병합테스트22(반포본동).xlsx')
origin.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,아동인구(0~9세),노인인구(65세이상),장애인_인구,세대당공원면적,보육시설수,세대당공원면적minmax,수변점수,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,612,2154,437,337.55,4,0.352898,0,32,0,3
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,488,1808,267,60.79,0,0.063554,0,27,0,12
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,108,626,110,956.51,3,1.000000,0,16,0,5
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,385,1814,311,0.00,3,0.000000,10,34,0,5
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,1033,3495,491,1.14,3,0.001192,10,54,0,5


In [3]:
origin.isna().sum()

시도명              0
시군구코드            0
시군구명             0
행정동코드            0
행정동코드_수정         0
읍면동명             0
생성일자             0
말소일자             0
지하철역_수           0
의료기관_수           0
사회복지시설_수         0
계(명)             0
면적 (k㎡)          0
구성비 (%)          0
행정동별_공원수         0
공공체육시설_수         0
문화시설_수           0
학교_수             0
사업체_수            0
개소 (공영+민영)       0
면수 (공영+민영)       0
하천및한강여부          0
아동인구(0~9세)       0
노인인구(65세이상)      0
장애인_인구           0
세대당공원면적          0
보육시설수            0
세대당공원면적minmax    0
수변점수             0
행정동별버스정류장수       0
행정동별유통업체수        0
공공자전거대여소수        0
dtype: int64

In [4]:
origin1=origin.fillna(0)

In [5]:
scaler=MinMaxScaler()

In [6]:
origin1.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수'],
      dtype='object')

In [7]:
df = origin1.copy()
df['의료기관minmax']=scaler.fit_transform(df[['의료기관_수']])
df['사회복지minmax']=scaler.fit_transform(df[['사회복지시설_수']])
df['보육시설minmax']=scaler.fit_transform(df[['보육시설수']])
df['노인인구minmax']=scaler.fit_transform(df[['노인인구(65세이상)']])
df['아동인구minmax']=scaler.fit_transform(df[['아동인구(0~9세)']])
df['장애인minmax']=scaler.fit_transform(df[['장애인_인구']])
df['면적minmax']=scaler.fit_transform(df[['면적 (k㎡)']])
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,행정동별버스정류장수,행정동별유통업체수,공공자전거대여소수,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,32,0,3,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.187952
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,27,0,12,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.080321
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,16,0,5,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.101205
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,34,0,5,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.163855
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,54,0,5,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.693976


In [8]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '의료기관minmax', '사회복지minmax', '보육시설minmax', '노인인구minmax',
       '아동인구minmax', '장애인minmax', '면적minmax'],
      dtype='object')

In [9]:
# 면적대비_거주하는_사회적_약자_수 파생지수(1) 생성
df['면적대비_거주하는_사회적_약자_수'] = (df['노인인구minmax'] + df['아동인구minmax'] + df['장애인minmax']) / df['면적minmax']
df.head()

# df['사회적약자_지원_인프라_밀집도'] = (df['의료기관minmax'] + df['사회복지minmax'] + df['보육시설minmax']) / (df['노인인구minmax'] + df['아동인구minmax'] + df['장애인minmax'])
# df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,행정동별유통업체수,공공자전거대여소수,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0,3,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.187952,2.438596
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0,12,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.080321,4.395563
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0,5,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.101205,1.149392
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0,5,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.163855,2.135894
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0,5,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.693976,0.996171


In [13]:
df['면적대비_거주하는_사회적_약자_수'].value_counts()

# '면적대비_거주하는_사회적_약자_수' 열에서 'inf' 값을 0으로 변경
df['면적대비_거주하는_사회적_약자_수'].replace([np.inf, -np.inf], 0, inplace=True)

df['면적대비_거주하는_사회적_약자_수minmax']=scaler.fit_transform(df[['면적대비_거주하는_사회적_약자_수']])
df

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,공공자전거대여소수,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,3,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.187952,2.438596,0.013697
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,12,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.080321,4.395563,0.024690
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,5,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.101205,1.149392,0.006456
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,5,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.163855,2.135894,0.011997
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,5,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.693976,0.996171,0.005595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,62,...,5,0.096875,0.222222,0.230769,0.510127,0.137894,0.294021,0.035341,26.655501,0.149723
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,80,...,6,0.125000,0.222222,0.282051,0.467674,0.157417,0.249589,0.038554,22.687002,0.127432
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,103,...,7,0.160938,0.407407,0.435897,0.968875,0.316833,0.544432,0.110843,16.511051,0.092742
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,1,...,4,0.001563,0.000000,0.000000,0.000000,0.000154,0.000000,0.055422,0.002774,0.000016


In [14]:
# 면적대비_거주하는_사회적_약자_수 파생지수(1) 생성
df['사회적약자_지원_인프라_밀집도'] = (df['의료기관minmax'] + df['사회복지minmax'] + df['보육시설minmax']) / df['면적대비_거주하는_사회적_약자_수minmax']
df.head()

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.187952,2.438596,0.013697,22.262225
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.080321,4.395563,0.024690,6.246513
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.101205,1.149392,0.006456,14.577024
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.163855,2.135894,0.011997,19.281214
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.693976,0.996171,0.005595,90.746521


In [15]:
df['사회적약자_지원_인프라_밀집도'].value_counts().sort_index()

사회적약자_지원_인프라_밀집도
0.059172      1
0.105912      1
0.172872      1
0.361605      1
0.506683      1
             ..
117.257482    1
127.010139    1
140.547466    1
185.242593    1
inf           2
Name: count, Length: 425, dtype: int64

In [16]:
df['사회적약자_지원_인프라_밀집도'].isna().sum()

0

In [17]:
df['사회적약자_지원_인프라_밀집도'].replace([np.inf, -np.inf], 0, inplace=True)
df

,시도명,시군구코드,시군구명,행정동코드,행정동코드_수정,읍면동명,생성일자,말소일자,지하철역_수,의료기관_수,...,의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,아동인구minmax,장애인minmax,면적minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도
0,서울특별시,11110,종로구,1111051500,11110515,청운효자동,20081101,0,0,11,...,0.017188,0.185185,0.102564,0.245223,0.094081,0.119035,0.187952,2.438596,0.013697,22.262225
1,서울특별시,11110,종로구,1111053000,11110530,사직동,19880423,0,2,75,...,0.117188,0.037037,0.000000,0.205630,0.075019,0.072408,0.080321,4.395563,0.024690,6.246513
2,서울특별시,11110,종로구,1111054000,11110540,삼청동,19880423,0,0,11,...,0.017188,0.000000,0.076923,0.070374,0.016603,0.029347,0.101205,1.149392,0.006456,14.577024
3,서울특별시,11110,종로구,1111055000,11110550,부암동,19880423,0,0,4,...,0.006250,0.148148,0.076923,0.206317,0.059185,0.084476,0.163855,2.135894,0.011997,19.281214
4,서울특별시,11110,종로구,1111056000,11110560,평창동,19880423,0,0,15,...,0.023438,0.407407,0.076923,0.398673,0.158801,0.133845,0.693976,0.996171,0.005595,90.746521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,서울특별시,11740,강동구,1174065000,11740650,성내제2동,19880423,0,3,62,...,0.096875,0.222222,0.230769,0.510127,0.137894,0.294021,0.035341,26.655501,0.149723,3.672561
422,서울특별시,11740,강동구,1174066000,11740660,성내제3동,19880423,0,1,80,...,0.125000,0.222222,0.282051,0.467674,0.157417,0.249589,0.038554,22.687002,0.127432,4.938113
423,서울특별시,11740,강동구,1174068500,11740685,길동,20080707,0,1,103,...,0.160938,0.407407,0.435897,0.968875,0.316833,0.544432,0.110843,16.511051,0.092742,10.828355
424,서울특별시,11740,강동구,1174069000,11740690,둔촌제1동,19880423,0,1,1,...,0.001563,0.000000,0.000000,0.000000,0.000154,0.000000,0.055422,0.002774,0.000016,100.287255


In [18]:
df['사회적약자_지원_인프라_밀집도'].unique()

array([2.22622252e+01, 6.24651295e+00, 1.45770242e+01, 1.92812136e+01,
       9.07465213e+01, 6.69712844e-01, 9.09080984e-01, 3.06608237e+00,
       5.92152614e+01, 9.30682327e+00, 9.35556031e+00, 5.50178386e+00,
       6.61142350e-01, 1.05911558e-01, 0.00000000e+00, 7.08190866e-01,
       3.19513119e+01, 1.26408183e+01, 4.59273910e+01, 1.56454366e+01,
       5.87921208e+00, 3.56055088e+00, 9.92061863e+00, 1.44877579e+00,
       2.08884700e+00, 1.64953139e+00, 5.06682659e-01, 1.00383538e+00,
       1.72872151e-01, 5.38623400e-01, 3.16326143e+00, 7.03129816e+00,
       5.93426782e+00, 1.86715686e+01, 4.76129159e+00, 3.55421830e+00,
       4.43859945e+00, 3.95436694e+00, 3.61604781e-01, 1.93559253e+01,
       1.35697666e+01, 5.60949665e+00, 4.66298014e+00, 2.37056369e+00,
       2.11767954e+01, 1.50727135e+01, 2.10527583e+00, 1.29344406e+00,
       6.26634332e+00, 8.72320201e+00, 7.34217075e+00, 3.50571860e+00,
       1.11598015e+00, 1.70265702e+00, 1.75467098e+00, 3.18591530e+00,
      

In [19]:
df.columns

Index(['시도명', '시군구코드', '시군구명', '행정동코드', '행정동코드_수정', '읍면동명', '생성일자', '말소일자',
       '지하철역_수', '의료기관_수', '사회복지시설_수', '계(명)', '면적 (k㎡)', '구성비 (%)',
       '행정동별_공원수', '공공체육시설_수', '문화시설_수', '학교_수', '사업체_수', '개소 (공영+민영)',
       '면수 (공영+민영)', '하천및한강여부', '아동인구(0~9세)', '노인인구(65세이상)', '장애인_인구',
       '세대당공원면적', '보육시설수', '세대당공원면적minmax', '수변점수', '행정동별버스정류장수', '행정동별유통업체수',
       '공공자전거대여소수', '의료기관minmax', '사회복지minmax', '보육시설minmax', '노인인구minmax',
       '아동인구minmax', '장애인minmax', '면적minmax', '면적대비_거주하는_사회적_약자_수',
       '면적대비_거주하는_사회적_약자_수minmax', '사회적약자_지원_인프라_밀집도'],
      dtype='object')

In [20]:
df1=df.filter(['시군구코드','시군구명','행정동코드','행정동코드수정','읍면동명','의료기관수','사회복지시설수','보육시설','노인인구','장애인인구','아동인구','면적 (k㎡)','의료기관minmax','사회복지minmax','보육시설minmax','노인인구minmax','장애인minmax','아동인구minmax','면적대비_거주하는_사회적_약자_수','면적대비_거주하는_사회적_약자_수minmax','사회적약자_지원_인프라_밀집도'])
df1.head()

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,장애인minmax,아동인구minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도
0,11110,종로구,1111051500,청운효자동,2.57,0.017188,0.185185,0.102564,0.245223,0.119035,0.094081,2.438596,0.013697,22.262225
1,11110,종로구,1111053000,사직동,1.23,0.117188,0.037037,0.000000,0.205630,0.072408,0.075019,4.395563,0.024690,6.246513
2,11110,종로구,1111054000,삼청동,1.49,0.017188,0.000000,0.076923,0.070374,0.029347,0.016603,1.149392,0.006456,14.577024
3,11110,종로구,1111055000,부암동,2.27,0.006250,0.148148,0.076923,0.206317,0.084476,0.059185,2.135894,0.011997,19.281214
4,11110,종로구,1111056000,평창동,8.87,0.023438,0.407407,0.076923,0.398673,0.133845,0.158801,0.996171,0.005595,90.746521


In [21]:
df1['사회적약자_지원_인프라_밀집도minmax']=scaler.fit_transform(df1[['사회적약자_지원_인프라_밀집도']])
df1

,시군구코드,시군구명,행정동코드,읍면동명,면적 (k㎡),의료기관minmax,사회복지minmax,보육시설minmax,노인인구minmax,장애인minmax,아동인구minmax,면적대비_거주하는_사회적_약자_수,면적대비_거주하는_사회적_약자_수minmax,사회적약자_지원_인프라_밀집도,사회적약자_지원_인프라_밀집도minmax
0,11110,종로구,1111051500,청운효자동,2.57,0.017188,0.185185,0.102564,0.245223,0.119035,0.094081,2.438596,0.013697,22.262225,0.120179
1,11110,종로구,1111053000,사직동,1.23,0.117188,0.037037,0.000000,0.205630,0.072408,0.075019,4.395563,0.024690,6.246513,0.033721
2,11110,종로구,1111054000,삼청동,1.49,0.017188,0.000000,0.076923,0.070374,0.029347,0.016603,1.149392,0.006456,14.577024,0.078692
3,11110,종로구,1111055000,부암동,2.27,0.006250,0.148148,0.076923,0.206317,0.084476,0.059185,2.135894,0.011997,19.281214,0.104086
4,11110,종로구,1111056000,평창동,8.87,0.023438,0.407407,0.076923,0.398673,0.133845,0.158801,0.996171,0.005595,90.746521,0.489879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,11740,강동구,1174065000,성내제2동,0.67,0.096875,0.222222,0.230769,0.510127,0.294021,0.137894,26.655501,0.149723,3.672561,0.019826
422,11740,강동구,1174066000,성내제3동,0.71,0.125000,0.222222,0.282051,0.467674,0.249589,0.157417,22.687002,0.127432,4.938113,0.026658
423,11740,강동구,1174068500,길동,1.61,0.160938,0.407407,0.435897,0.968875,0.544432,0.316833,16.511051,0.092742,10.828355,0.058455
424,11740,강동구,1174069000,둔촌제1동,0.92,0.001563,0.000000,0.000000,0.000000,0.000000,0.000154,0.002774,0.000016,100.287255,0.541383


In [22]:
df1.to_excel('사회적약자_지원_인프라_밀집도1.xlsx',index=False)